In [ ]:
import matplotlib.pylab as plt
import numpy as np

import sys

import datetime as dt
import pandas as pd

import seaborn as sns

In [ ]:


################################################################################
def sort_courses(courses):

    depts = []
    numbers = []
    unique_courses = []

    for course in courses:
        depts.append(course['id'].split()[0])
        numbers.append(int(course['id'].split()[1]))
        
        identifier = (course['id'],course['type'])
        if identifier not in unique_courses:
            unique_courses.append(identifier)

    idx = np.argsort(numbers)

    #print(idx)

    sorted_courses = []
    for i in idx:
        #print(courses[i])
        sorted_courses.append(courses[i])

    #print(unique_courses)
    unique_courses.sort()
    #print(unique_courses)

    new_courses = []
    for i,c in enumerate(unique_courses):

        new_courses.append({'identifier':c,'instances':[]})

        for course in sorted_courses:
            
            identifier = (course['id'],course['type'])
            if identifier == c:
                new_courses[i]['instances'].append(course)


    return new_courses

################################################################################

################################################################################
def term2date(term):

    # By hand
    #year = 2000 + int(term[1:])
    # Read in
    year = int(term.split()[-1])
    monthstart = 1
    monthend = 5
    daystart = 15
    dayend = 1
    # By hand
    #if term[0] == 'F':
    # Read in
    if term.find('Fall')>=0:
        monthstart = 9
        monthend = 12
        daystart = 1
        dayend = 15
    #day = 15
    # By hand
    #year = 2000 + int(term[1:])
    # Read in
    year = int(term.split()[-1])
    start = dt.datetime(year, monthstart, daystart, 0, 0)
    end = dt.datetime(year, monthend, dayend, 0, 0)

    return start,end

################################################################################




In [ ]:
################################################################################
# Reading in from what we download from Banner
################################################################################
df = pd.read_html('courses_taught_downloads.html')
df = df[-1]
#df['name'] = df['Course Title']
#df['id'] = df['Course']
#df['term'] = df['Associated Term']
courses = []
nc = len(df)
for i in range(nc):
    course = {}
    course['id'] = df['Course'].iloc[i]
    course['name'] = df['Course Title'].iloc[i]
    course['term'] = df['Associated Term'].iloc[i]
    course['year'] = int(df['Associated Term'].iloc[i].split()[-1])
    course['nstudents'] = 18
    if df['Course Title'].iloc[i].find('Lab')>=0:
        course['type'] = 1
    elif df['Course'].iloc[i].find('499')>=0:
        course['type'] = 2
    else:
        course['type'] = 0

    if course['year'] > 2015:
        continue
    courses.append(course)



courses = sort_courses(courses)

#print(courses)

fig, ax = plt.subplots(figsize=(18,12))
course_names = []
for i,course in enumerate(courses):
    #identifier = course['identifier'][0]
    identifier = '{0} - {1}'.format(course['instances'][0]['name'],course['identifier'][0])
    course_names.append(identifier)
    #print(identifier)
    for instance in course['instances']:
        start,end = term2date(instance['term'])
        xranges = [(start,end-start)]
        yrange = (i,1.0)
        # Plot the broken horizontal bars
        #print(xranges)
        fc = 'blue'
        if instance['type']==1:
            fc = 'orange'
        elif instance['type']==2:
            fc = 'red'
        plt.broken_barh(xranges, yrange, facecolors=fc)

#course_names.reverse()
#print(course_names)
y_pos = np.arange(0,len(courses),1) + 0.5
ax.set_yticks(y_pos)
ax.set_yticklabels(course_names)
plt.grid(axis='y')

plt.xlim(dt.datetime(2012,1,1,1,1))

sabbatical_start = dt.datetime(2018,9,1)
sabbatical_end = dt.datetime(2019,9,1)

plt.plot([sabbatical_end,sabbatical_end],[0,y_pos[-1]+3],'k--',label='End of sabbatical')
plt.plot([sabbatical_start,sabbatical_start],[0,y_pos[-1]+3],'k-',label='Start of sabbatical')



ax.xaxis_date()
plt.xticks(fontsize=18)

plt.legend(fontsize=24)

plt.tight_layout()

########################################################



In [ ]:
df

In [ ]:
#courses

In [ ]:
df.sort_values(by='Associated Term')

In [ ]:
# Add some columns
#df.columns
year = []
term = []
start = []
end = []
lab = []
independent_study = []

ncourses = len(df)

for i in range(ncourses):
    aterm = df.iloc[i]['Associated Term']
    s,e = term2date(aterm)
    #print(start,end)
    year.append(s.year)
    term.append(aterm.split()[0])
    start.append(s)
    end.append(e)
    
    if df.iloc[i]['Course Title'].find('Lab')>=0 or df.iloc[i]['Course Title'].find('lab')>=0:
        lab.append('Lab')
    elif df.iloc[i]['Course'].find('499')>=0 or df.iloc[i]['Course'].find('497')>=0 or \
    df.iloc[i]['Course'].find('HNRS')>=0:
        lab.append("IS")
    else:
        lab.append('Lecture')
        
    if df.iloc[i]['Course'].find('499')>=0 or df.iloc[i]['Course'].find('497')>=0 or \
    df.iloc[i]['Course'].find('HNRS')>=0:
        independent_study.append("IS")
    else:
        independent_study.append("NO")
    
    #print(df.iloc[i]['Instructional Method'])
        

        
df['year'] = year
df['term'] = term
df['start'] = start
df['end'] = end
df['type'] = lab
df['independent study'] = independent_study


df

In [ ]:
#df['independent study']

In [ ]:
mask = df['independent study'] != 'IS'
df[mask][['Course','Course Title']].value_counts()
#df[['type']].value_counts()

In [ ]:
c_dict = {'NSF':'#E64646', 'CURCA':'#E69646', 'ENG':'#34D05C', 'APS':'#34D0C3', 'AAPT':'#3475D0'}

#plt.figure(figsize=(12,8))
#df.plot.scatter(x='start',y='amount',ax=plt.gca(), s=1)

for i in range(len(df)):

    x = df.iloc[i]['start']
    y = df.iloc[i]['amount']
    
    agency = df.iloc[i]['funding_agency']
    funded = df.iloc[i]['funded']
    dur = df.iloc[i]['duration']
    #print(dur.days)
    
    color = c_dict[agency]
    alpha=1.0
    hatch = ''
    linecolor = color
    facecolor = color
    linewidth = 5
    
    if funded==False:
        alpha = 0.8
        hatch = '/'
        linecolor = color
        facecolor = 'black'
        linewidth=5

    
    width = dt.timedelta(days=120)

    bar = plt.Rectangle((x,1),width,y,color=facecolor,alpha=alpha,ec=linecolor,linewidth=linewidth,hatch=hatch)
    plt.gca().patches.append(bar)

#plt.ylabel('Amount ($)',fontsize=24)
#plt.xlabel('Start date of grant',fontsize=24)
#plt.xticks(fontsize=14);
#plt.yticks(fontsize=14);